In [28]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
from cpmpy import * # pip3 install cpmpy
from cpmpy.solvers import CPM_ortools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# helper function to enumerate all models, for a list of key variables
def enumerate_all(model, variables):
    model = CPM_ortools(model) # this speeds up repeated solving
    while model.solve():
        print(variables.value())
        model += ~all(variables == variables.value())

What I think is the underlying model:

In [34]:
# returns (model, variables)
def manual_type02(size=7):
    v = intvar(1,size, shape=size)
    
    m = Model()
    
    # all_pairs: v[i] != v[j]
    m += AllDifferent(v)
    
    # subsequent_pairs: abs(diff(v[i],v[j])) >= 2
    for i in range(1,len(v)):
        m += abs(v[i-1] - v[i]) >= 2

    return (m,v)

In [35]:
# lets enumerate a few
(m,v) = manual_type02(size=4)
enumerate_all(m,v)

[2 4 1 3]
[3 1 4 2]


Example checking the model

In [46]:
# returns % correctly true
def check_solutions(m, sols, verbose=False):
    # are there any solutions given?
    if len(sols) == 0:
        print("No solutions to check")
        return 1.0
        
    sats = []
    for sol in sols:
        sol = sol['list']
        m2 = Model([c for c in m.constraints]) # euh, CPMpy needs a model.copy()...
        m2 += (v == sol)
        sat = m2.solve()
        sats.append(sat)
        
        if verbose:
            if sat:
                print(f"Sol {sol} indeed satisfies")
            else:
                print(f"!!! Sol {sol} does not satisfy after all")
    return sum(sats)/len(sats)
        
        
    if m2.solve():
        print(f"Sol {sol} indeed satisfies")
    else:
        print(f"Sol {sol} does not satisfy after all")

In [47]:
data = json.load(open('instances/type02/instance0.json'))
(m,v) = manual_type02(size=data['size'])
check_solutions(m, data['solutions'], verbose=True)

Sol [3, 1, 4, 2] indeed satisfies
Sol [2, 4, 1, 3] indeed satisfies


1.0

In [48]:
data = json.load(open('instances/type02/instance7.json'))
(m,v) = manual_type02(size=data['size'])
check_solutions(m, data['solutions'])

1.0

In [49]:
data = json.load(open('instances/type02/instance9.json'))
(m,v) = manual_type02(size=data['size'])
check_solutions(m, data['solutions'])

No solutions to check


1.0

In [59]:
# returns % correctly false
def check_nonsolutions(m, sols, verbose=False):
    # are there any solutions given?
    if len(sols) == 0:
        print("No solutions to check")
        return 1.0
        
    sats = []
    for sol in sols:
        sol = sol['list']
        m2 = Model([c for c in m.constraints]) # euh, CPMpy needs a model.copy()...
        m2 += (v == sol)
        sat = m2.solve()
        sats.append(sat)
        
        if verbose:
            if sat:
                print(f"!!! non-sol {sol} does satisfies")
            else:
                pass
                #print(f"non-sol {sol} indeed does not satisfy")
    
    # return % correct unsat!!
    return (1- sum(sats)/len(sats))

In [60]:
data = json.load(open('instances/type02/instance0.json'))
(m,v) = manual_type02(size=data['size'])
check_nonsolutions(m, data['nonSolutions'], verbose=True)

1.0

In [61]:
data = json.load(open('instances/type02/instance7.json'))
(m,v) = manual_type02(size=data['size'])
check_nonsolutions(m, data['nonSolutions'])

0.998

# OK, curious... one is SAT?

In [62]:
check_nonsolutions(m, data['nonSolutions'], verbose=True)

!!! non-sol [7, 5, 3, 1, 10, 2, 6, 4, 8, 11, 9] does satisfies
!!! non-sol [9, 2, 6, 8, 3, 1, 10, 4, 11, 7, 5] does satisfies


0.998

Weird... did they make a mistake, or did we overlook something?

In [66]:
import glob
files = glob.glob("instances/type02/instance*.json")

for file in sorted(files):
    data = json.load(open(file))
    (m,v) = manual_type02(size=data['size'])
    print(file, check_nonsolutions(m, data['nonSolutions']))

instances/type02/instance0.json 1.0
instances/type02/instance1.json 1.0
instances/type02/instance2.json 1.0
instances/type02/instance3.json 1.0
instances/type02/instance4.json 0.990909090909091
instances/type02/instance5.json 1.0
instances/type02/instance6.json 0.999
instances/type02/instance7.json 0.998
No solutions to check
instances/type02/instance8.json 1.0
No solutions to check
instances/type02/instance9.json 1.0


OK, so there are either some rare cases which they got wrong, or I missed some exceptional detail...